# Triangulation Matting
### Import Packages

In [ ]:
import re
import os
import pylab
from PIL import Image
import numpy as np
from os import path
import matplotlib.pyplot as plt
import cv2
from numba import njit
import rawpy
from imutils import paths
import shutil
import glob

raw_jpg = "/home/jannes/uni/jk-masterarbeit/green_benchmark/raw_jpg"
postview = "/home/jannes/uni/jk-masterarbeit/green_benchmark/postview"


### Triangulation Matting Implementation

In [ ]:
def load_image(dir, name, reduce:int=None):
    im_path = path.join(dir, name)
    if ".ARW" in name:
        raw = rawpy.imread(im_path)
        arr = raw.postprocess(rawpy.Params(use_camera_wb=True, fbdd_noise_reduction=rawpy.FBDDNoiseReductionMode.Full))
    else:
        arr = np.asarray(Image.open(fp=im_path))
    if reduce is not None:
        arr = arr[::reduce,::reduce,:]
    return arr

@njit( parallel=True)
def triangulation_matting(img1, img1bg, img2, img2bg):
    img1delta = img1/255 - img1bg/255
    img2delta = img2/255 - img2bg/255

    b = np.concatenate((img1delta, img2delta), axis=-1)

    alpha = np.zeros(img1.shape[:2])
    colors = np.zeros(img1.shape)
    for r in range(len(img1)):
        for c in range(len(img1[0])):
            a = np.array([
                [1,0,0,-1/255*img1bg[r][c][0]],
                [0,1,0,-1/255*img1bg[r][c][1]],
                [0,0,1,-1/255*img1bg[r][c][2]],
                [1,0,0,-1/255*img2bg[r][c][0]],
                [0,1,0,-1/255*img2bg[r][c][1]],
                [0,0,1,-1/255*img2bg[r][c][2]],
            ])
            x = np.clip(np.linalg.pinv(a).dot(b[r][c]), 0, 1)
            colors[r,c] = np.array([x[0], x[1], x[2]])
            alpha[r,c] = x[3]

    alpha = np.expand_dims(alpha, axis=-1)
    return colors, alpha


### List Object dirs
Paste the directory name of a captured object and the corresponding background
directory name in the list of tuples.

In [ ]:
objects_path = "/home/jannes/uni/jk-masterarbeit/green_benchmark/raw_jpg"
objects = [
    ("cactus_20230221-230300", "cactus_bg_20230221-230416"),
    ("elephant_20230221-231530", "elephant_bg_20230221-232319"),
    ("teddy_20230222-011042", "teddy_bg_20230222-011506"),
    ("belt_20230222-010233", "belt_bg_20230222-010420"),
    ("headphones_20230222-004515", "headphones_bg_20230222-005024"),
    ("waterbottle_20230222-002305", "waterbottle_bg_20230222-002402"),
    ("palm_20230222-000910", "palm_bg_20230222-001051"),
    ("palm_20230222-000612", "palm_bg_20230222-001051"),
    ("thermos_20230221-235201", "thermos_bg_20230221-235321"),

    ("doll_20230228-230757", "doll_bg_20230228-230927"),
    ("doll_f3_2_20230301-000845", "doll_f3_2_bg_20230301-001005"),
    ("doll_f5_6_20230301-001322", "doll_f5_6_bg_20230301-001400"),
    ("doll_f8_20230301-001736", "doll_f8_bg_20230301-001850"),
    ("curtain_20230301-010421", "curtain_bg_20230301-010555"),

    ("10er_20230308-003805", "10er_bg_20230308-003846"),
    ("apple_20230308-004012", "apple_bg_20230308-004049"),
    ("dogs_20230308-004828", "dogs_bg_20230308-004915"),
    ("horse_20230308-005709", "horse_bg_20230308-005751"),
    ("people_20230308-010809", "people_bg_20230308-011037"),
    ("people_20230308-010950", "people_bg_20230308-011037"),
    ("people2_20230308-011552", "people2_bg_20230308-011703"),
    ("noodles_20230308-014234", "noodles_bg_20230308-014807"),
    ("noodles_colors_20230308-014714", "noodles_bg_20230308-014807"),
    ("colors_20230308-020041", "noodles_bg_20230308-014807"),
    ("colors2_20230308-020652", "colors2_bg_20230308-020859"),
    ("colors3_20230308-021435", "colors2_bg_20230308-020859"),

    ("car_20230525-024456", "car_bg_20230525-024543"),
    ("leaves_20230525-025257", "leaves_bg_20230525-025540"),
    ("leaves2_20230525-025435", "leaves_bg_20230525-025540"),
    ("fox_spill_20230525-031300", "fox_bg_20230525-031528"),
    ("fox_20230525-031354", "fox_bg_20230525-031528"),
    # ("", ""),
]

### Execute Triangulation Matting

In [ ]:
img1_name =   "green96"
img1bg_name = ""
img2_name =   "blue96"
img2bg_name = ""
img_black_name = "black"

for object, bg in objects:
    images = paths.list_images(path.join(objects_path, object))
    images_bg = paths.list_images(path.join(objects_path, bg))
    img1 = None
    img1bg = None
    img2 = None
    img2bg = None
    img_black = None
    for img in images:
        img = path.split(img)[1]
        if img1_name in img:
            img1 =   load_image(objects_path, f"{object}/{img}", reduce=None)
        if img2_name in img:
            img2 =   load_image(objects_path, f"{object}/{img}", reduce=None)
        if img_black_name in img:
            img_black = load_image(objects_path, f"{object}/{img}", reduce=None)/255

    for img_bg in images_bg:
        img_bg = path.split(img_bg)[1]
        if img1_name in img_bg:
            img1bg = load_image(objects_path, f"{bg}/{img_bg}", reduce=None)
        if img2_name in img_bg:
            img2bg = load_image(objects_path, f"{bg}/{img_bg}", reduce=None)

    colors, alpha = triangulation_matting(img1, img1bg, img2, img2bg)

    gt = np.concatenate((img_black, alpha), axis=-1)
    out = np.concatenate((colors, alpha), axis=-1)

    save_dir = path.join(objects_path, object, "matting")
    os.makedirs(save_dir, exist_ok=True)
    pylab.imsave(path.join(save_dir, "out.png"), out)
    pylab.imsave(path.join(save_dir, "gt.png"), gt)
    pylab.imsave(path.join(save_dir, "alpha.png"), np.squeeze(alpha, axis=-1), cmap=pylab.cm.gray)

    print(f"Finished {object} ")

### Alpha Map Clipping

In [ ]:
for object, _ in objects:
    matting_dir = path.join(objects_path, object, "matting")
    alpha_path = path.join(matting_dir, "alpha.png")
    alpha = np.asarray(Image.open(fp=alpha_path))[:,:,0]
    alpha_corr = np.where(alpha >= 246, 255, alpha)
    alpha_corr = np.where(alpha_corr <= 8, 0, alpha_corr)

    pylab.imsave(path.join(matting_dir, "alpha_clip.png"), alpha_corr, cmap=pylab.cm.gray)
    print(f"Finished {object} ")



### Edited Ground Truth
After editing the alpha maps, the complete ground truth can be created.

In [ ]:
for object, _ in objects:
    matting_dir = path.join(objects_path, object, "matting")
    alpha_edit_path = path.join(matting_dir, "alpha_edit.png")
    gt_path = path.join(matting_dir, "gt.png")
    if path.isfile(alpha_edit_path):
        alpha = np.asarray(Image.open(fp=alpha_edit_path))[:,:,0]
        gt = np.asarray(Image.open(fp=gt_path))
        gt_edit = np.copy(gt)
        gt_edit[:,:,3] = alpha

        pylab.imsave(path.join(matting_dir, "gt_edit.png"), gt_edit)
        print(f"Finished {object} ")
    else:
        print(alpha_edit_path, "not found")



### Crop and finalize Dataset


In [ ]:
crop_images = {
    "cactus": 0.7,
    "elephant": 0.8,
    "belt": 0.8,
    "10er": 0.6,
    "apple": 0.6,
    "dogs": 0.7,
    "horse": 0.7,
    "people": 0.5,
    "car": 0.5,
}

red = "red"
green = "green"
blue = "blue"
black = "black"
alpha = "alpha"
gt = "ground_truth"

benchmark_path = "/home/jannes/uni/jk-masterarbeit/green_benchmark/benchmark"
os.makedirs(path.join(benchmark_path, alpha), exist_ok=True)
n = len(os.listdir(path.join(benchmark_path, alpha)))
object_name_re = re.compile(r"[^/]+(?=_2023)")

def get_new_image_name(path, ext):
    object_name = object_name_re.search(path).group()
    return f"{n}_{object_name}.png"

for object, _ in objects:
    print(f"Copy {object}..")
    dir = path.join(objects_path, object)
    red_path = glob.glob(path.join(dir, "*red*.JPG"))[0]
    green_path = glob.glob(path.join(dir, "*green*.JPG"))[0]
    blue_path = glob.glob(path.join(dir, "*blue*.JPG"))[0]
    black_path = glob.glob(path.join(dir, "*black*.JPG"))[0]
    alpha_path = glob.glob(path.join(dir, "matting", "alpha_edit.png"))[0]
    gt_path = glob.glob(path.join(dir, "matting", "gt_edit.png"))[0]
    img_copy = [
        (red_path, path.join(benchmark_path, red, get_new_image_name(red_path, ".JPG"))),
        (green_path, path.join(benchmark_path, green, get_new_image_name(green_path, ".JPG"))),
        (blue_path, path.join(benchmark_path, blue, get_new_image_name(blue_path, ".JPG"))),
        (black_path, path.join(benchmark_path, black, get_new_image_name(black_path, ".JPG"))),
        (alpha_path, path.join(benchmark_path, alpha, get_new_image_name(alpha_path, ".png"))),
        (gt_path, path.join(benchmark_path, gt, get_new_image_name(gt_path, ".png"))),
    ]

    for src, dest in img_copy:
        os.makedirs(os.path.dirname(dest), exist_ok=True)
        image = Image.open(src)
        if ".JPG" in src:
            image = image.convert('RGB')

        for name, ratio in crop_images.items():
            if name in src:
                    width, height = image.size
                    new_width = int(width * ratio)
                    new_height = int(height * ratio)

                    crop_width = new_width
                    crop_height = new_height

                    x = int((width - crop_width) / 2)
                    y = height - crop_height

                    crop_box = (x, y, x + crop_width, y + crop_height)
                    image = image.crop(crop_box)

        image.save(dest)

    n = n + 1